In [1]:
import gym
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Get Taxi-v3 environment
env = gym.make('Taxi-v3')

model = Sequential()
model.add(Dense(50, input_dim=env.observation_space.n, activation='relu'))
model.add(Dense(env.action_space.n, activation='linear'))
model.compile(loss='mse', optimizer=Adam())

# Gotta adjust those cause suboptimal atm
epsilon = 0.5
epsilon_decay = 0.5
gamma = 0.95
num_episodes = 1 

for episode in range(num_episodes):
    state = env.reset()
    state_one_hot = np.zeros([1, env.observation_space.n])
    state_one_hot[0, state[0]] = 1

    done = False
    iters = 1
    while not done:
        print("Episode: {}, Iteration: {}".format(episode, iters))
        # Epsilon-greedy action selection
        if np.random.rand() <= epsilon:
            # Explore
            action = env.action_space.sample()
        else:
            # Exploit
            action = np.argmax(model.predict(state_one_hot))

        next_state, reward, done, _, _ = env.step(action)
        next_state_one_hot = np.zeros([1, env.observation_space.n])
        next_state_one_hot[0, next_state] = 1

        # Update Q-value
        target = model.predict(state_one_hot)
        if done:
            target[0][action] = reward
        else:
            target[0][action] = reward + gamma * np.max(model.predict(next_state_one_hot))

        # Train the model
        model.fit(state_one_hot, target, epochs=1, verbose=0)
        
        state_one_hot = next_state_one_hot
        iters += 1

    # Decay the epsilon value
    if epsilon > 0.01:
        epsilon *= epsilon_decay



Episode: 0, Iteration: 1
1/1 [==============================] - 0s 16ms/step


c:\Users\szomo\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 0, Iteration: 2
1/1 [==============================] - 0s 19ms/step
Episode: 0, Iteration: 3
1/1 [==============================] - 0s 27ms/step
Episode: 0, Iteration: 4
1/1 [==============================] - 0s 22ms/step
Episode: 0, Iteration: 5
1/1 [==============================] - 0s 23ms/step
Episode: 0, Iteration: 6
1/1 [==============================] - 0s 16ms/step
Episode: 0, Iteration: 7
1/1 [==============================] - 0s 20ms/step
Episode: 0, Iteration: 8
1/1 [==============================] - 0s 16ms/step
Episode: 0, Iteration: 9
1/1 [==============================] - 0s 13ms/step
Episode: 0, Iteration: 10
1/1 [==============================] - 0s 25ms/step
Episode: 0, Iteration: 11
1/1 [==============================] - 0s 17ms/step
Episode: 0, Iteration: 12
1/1 [==============================] - 0s 23ms/step
Episode: 0, Iteration: 13
1/1 [==============================] - 0s 23ms/step
Episode: 0, Iteration: 14
1/1 [==============================] - 0s 13ms

In [4]:
import matplotlib.pyplot as plt

num_test_episodes = 100
rewards = []

# Test the trained agent
for episode in range(num_test_episodes):
    state = env.reset()
    state = np.reshape(state, [1, env.observation_space.n])
    episode_reward = 0
    done = False
    while not done:
        # Exploit learned values
        action = np.argmax(model.predict(state))
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, env.observation_space.n])
        state = next_state
        episode_reward += reward
    rewards.append(episode_reward)

plt.plot(rewards)
plt.title('Average reward per episode')
plt.xlabel('Episode')
plt.ylabel('Average Reward')
plt.show()


c:\Users\szomo\AppData\Local\Programs\Python\Python310\lib\site-packages\gym\envs\toy_text\taxi.py:282: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("Taxi-v3", render_mode="rgb_array")
  logger.warn(


AssertionError: Something went wrong with pygame. This should never happen.